In [1]:
import sys
sys.path+=['../']

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from multiprocessing import Pool
import param
from IPython.utils import io
import gc

In [3]:
%time sra_dump_df=pd.read_csv(param.full_meta_dir)

<string>:2: DtypeWarning: Columns (5,6,25,26) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 39.1 s, sys: 3.96 s, total: 43 s
Wall time: 40.2 s


#### check if data is in SRA

In [4]:
allProcessedFnames=pd.Series(os.listdir(param.count_out_dir),dtype=np.str)

In [5]:
postfix_m=allProcessedFnames.str.contains('.abundance.tsv.gz$')
processedRnaseqSrr=allProcessedFnames[postfix_m].str.split('.').str[0].unique()

In [6]:
len(processedRnaseqSrr)

454926

In [7]:
sra_dump_df['Skymap_TranscriptCount_Processed']=sra_dump_df['Run'].isin(processedRnaseqSrr)

In [8]:
m_processed=sra_dump_df['Skymap_TranscriptCount_Processed']

### select the species

In [9]:
faBaseDir='/cellar/users/btsui/Data/ensembl/release/cdna/'

In [10]:
nProcess=32

In [11]:
metricToDtypeDict={'est_counts':np.uint32,'tpm':np.float32}

In [12]:
from itertools import product

In [13]:
specieMetricPairs=list(product( param.supporting_species,metricToDtypeDict.keys()))
#selectedSpecies=

In [ ]:
#%%capture Stdout
#selectedSpecies ,countMetric=('Canis_familiaris', 'tpm')  
ignoreLastNDigitsForChunking=5
for selectedSpecies ,countMetric in specieMetricPairs:
    
    #identify samples
    print (selectedSpecies ,countMetric)
    m_specie=sra_dump_df['new_ScientificName']==selectedSpecies
    m=m_processed&m_specie
    sra_dump_df_sub=sra_dump_df[m]
    sra_dump_df_in=sra_dump_df_sub#.head(n=10)
    nTotal=sra_dump_df_sub.shape[0]
    print ('# of samples to merge:',nTotal)
    inSrrs=sra_dump_df_in['Run'].unique()
    ### identify the datatype in consideration. 
    myDtype=metricToDtypeDict[countMetric]

    def parseOne(inSrr):
        abundanceDir=param.count_out_dir+'{}.abundance.tsv.gz'.format(inSrr)
        tmpDf=pd.read_csv(abundanceDir,sep='\t')
        tmpDf2=tmpDf.set_index('target_id')[countMetric].astype(myDtype)
        return tmpDf2
    #do the merge
    inSrrS=pd.Series(inSrrs)
    #each prefix is a Chunk ID
    groupRunDf=pd.DataFrame({'Prefix':inSrrS.str[:-ignoreLastNDigitsForChunking],'Run':inSrrS})
    print ("n chunks:",groupRunDf['Prefix'].nunique())
    for Prefix,subRunS in groupRunDf.groupby('Prefix')['Run']:
        with Pool(nProcess) as p:
            myL=list(tqdm( p.imap(parseOne,subRunS),total=len(subRunS)))
        #3 mins per chunk
        mergedDf=pd.concat(myL,axis=1,keys=subRunS.values,copy=False)
        outputDir='/nrnb/users/btsui/Data/all_seq/rnaseq_merged_chunks/{}.transcript.{}.{}.pickle'.format(
            selectedSpecies,countMetric,Prefix)
        mergedDf.to_pickle(outputDir)
        del (mergedDf, myL)
        gc.collect()
    

Homo_sapiens est_counts
# of samples to merge: 192981
n chunks: 86


100%|██████████| 1804/1804 [02:03<00:00, 14.56it/s]


In [ ]:
asdfaf

# scratch

In [ ]:
!ls -laht /nrnb/users/btsui/Data/all_seq/rnaseq_merged_chunks/